# Diameter Data

This file is part of the Glaucoma Phenotype ML Estimation project.

 Glaucoma Phenotype ML Estimation is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.


The Glaucoma Phenotype ML Estimation project is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with the Glaucoma Phenotype ML Estimation project.  If not, see <http://www.gnu.org/licenses/>.


In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [10]:
# imports
import pickle as pkl
import os
from pathlib import Path
from multiprocessing import Pool, cpu_count
import numpy as np
import pandas as pd
import zipfile
#import imageio
import matplotlib.pyplot as plt
import seaborn as sns
from fastai.vision import *
from PIL import Image
from IPython.display import display

from multiprocessing import freeze_support
from multiprocessing import Pool
from multiprocessing import cpu_count
from functools import partial
import threading
sys.setrecursionlimit(5000)
from fastai.distributed import *
from glaucoma.helpers.glaucoma_helpers import is_number, parse_files, return_uniq_index, return_duplicates, plot_idx, split_unzipped

In [15]:
# split up unzipped files
unzipped_head, unzipped_tail = split_unzipped(unzipped_files)

In [16]:
unzipped_check = [unzipped_head[i] +"/"+unzipped_tail[i] for i in range(len(unzipped_head))]

In [14]:
##### PLEASE SET AS REQUIRED########
WORKING_DIR = Path(os.getcwd())
DATA_DIR = WORKING_DIR / 'data'
META_DIR = DATA_DIR / 'metadata'
TRAIN_DIR =DATA_DIR / "train"
UKBB_DIR = DATA_DIR / 'retinal_images/UKBB'
CROP_DIR = DATA_DIR/'retinal_images/cropped_UKBB'
UNZIPPED_DIR = DATA_DIR / 'unzipped'

In [15]:
uncropped_full = parse_files(UKBB_DIR)
uncropped_split = [os.path.split(f) for f in uncropped_full]

In [20]:
uncropped_dict = { h[1]:h[0] for h in uncropped_split}

In [22]:
uk_df = pd.read_csv(UKBB_DIR / 'clinical_gradings' / 'df_VCDR_DISC_grade_result_merge_2020_July_15_chunk_1_50_71_88 - df_VCDR_DISC_grade_result_merge_2020_July_15_chunk_1_50_71_88.csv.csv'  )

In [23]:
v2_meta_df = pkl.load(open(WORKING_DIR / 'v2_meta_df.pkl','rb'))

In [25]:
v2_meta_df[["coord1","coord2"]] = v2_meta_df.area.str.split(":",expand = True).loc[:,0:1]

## Looking at diameter data

In [28]:
def cal_radius(p1, p2):
    if isinstance(p1,str):
        p1 = tuple(map(int,p1.split(",")))
    if isinstance(p2,str):
        p2 = tuple(map(int,p2.split(",")))
    r2 = np.power(p2[0]- p1[0],2) + np.power(p2[1] - p1[1],2)
    return np.sqrt(r2)

In [64]:
#original size = (2048, 1536)
def diameter_visualise(file_path ,coord1, coord2, uncropped = False):
    #pull uncrooped 
    if uncropped:
        h,t = os.path.split(file_path)
        t = t[:-4] + ".png"
        file_path = uncropped_dict[t] +"/" + t
        
    
    im = plt.imread(file_path)
    implot = plt.imshow(im,origin = "upper")
    print("coord 1:" +str(coord1) +" coord 2:" +str(coord2) )
    #plot points
    x_len = im.shape[1]
    y_len = im.shape[0]

    print(x_len, y_len)
    #x_r =  x_len * (1 +12/770) /  770
    #y_r = (y_len / (583)) *6/5 
    
    x_r = x_len / 665
    y_r =y_len / float(665 / 1.35)
    x_shift = 0
       
    
    x1 =  (coord1[0] + x_shift ) * x_r 
    x2 =  (coord2[0] + x_shift) * x_r 
    
    y1 = (coord1[1] ) * y_r 
    y2 = (coord2[1]) * y_r 
    
    points =[[ x_len -x1,x_len- x2],[y1,y2]]
    points2 =[[x1,x2],[y1,y2]]
    points3 =[[x1,x2],[y_len-y1,y_len -y2]]
    points4 =[[x_len -x1,x_len - x2],[y_len - y1,y_len - y2]]

    plt.scatter(*points2)
    plt.plot(*points2,"b")
    #plt.plot(*points3)
    #plt.plot(*points4)
    
    

In [ ]:
v2_meta_df[v2_meta_df['file_name'] == '1021701_21015_0_0.jpg']

In [95]:
def plot_num(num):
    key = v2_meta_df.loc[num,:]
    idx = unzipped_tail.index(key["file_name"])
    print(key)
    im = diameter_visualise(unzipped_files[idx],tuple(map(int,key["coord1"].split(","))),tuple(map(int,key["coord2"].split(","))), False)

#idx = unzipped_tail.index('4821158_21015_0_0.jpg')
#im = diameter_visualise(unzipped_files[idx],(222,186),(218,331))


In [ ]:
v2_meta_df[v2_meta_df['file_name'] == '4969460_21015_0_0.jpg']

## building model with point data

In [111]:
L_only = [f for f in list(v2_meta_df['batch_name'].unique()) if f[0] == 'L']
r = re.compile("L_[1-9][1-9]_")
L_only = list(filter(r.match, L_only)) # Read Note


This function rescales the diameter points into the correct pixel coordinates, an artifact from the labeling program

In [130]:
def rescale_coord(image, p):
    #idx = unzipped_tail.index(image)
    #im = plt.imread(unzipped_files[idx])
    #x_len = im.shape[1]
    #y_len = im.shape[0]
    
    x_len, y_len = 1040, 800
    p = tuple(map(int,p.split(",")))
    
    x_r =  x_len /665
    y_r = (y_len / (665/1.35))
    
    x_scaled = int(np.round(p[0] * x_r))
    y_scaled = int(np.round(p[1] * y_r))
    
    return (x_scaled,y_scaled)

In [125]:
v2_used = v2_meta_df[v2_meta_df['batch_name'].isin(L_only)]

In [ ]:
#set the rescaled coordinates
v2_used['coord1_rs'] = v2_used.apply(lambda x: rescale_coord(x.file_name, x.coord1), axis=1)
v2_used['coord2_rs'] = v2_used.apply(lambda x: rescale_coord(x.file_name, x.coord2), axis=1)

## Build the method for getting points

### Move data

In [136]:
#remove ungraded imaiges
v2_graded = v2_used[~v2_used.coord1.str.contains('-1') ]

In [147]:
#move all gradable images across
for f_name in v2_graded['file_name']:
    idx = unzipped_tail.index(f_name)
    f = unzipped_files[idx]
    shutil.copy(f, DATA_DIR / 'diameter' /f_name)

### Build dataset

In [139]:
def coord_str_tup(string):
    return(tuple(map(int,string.split(','))))

In [150]:
def average_coords(df):
    coords1 = [[],[]]
    coords2 = [[],[]]
    for coord in df['coord1_rs']:
        tup = coord
        coords1[0].append(tup[0])
        coords1[1].append(tup[1])
    
    for coord in df['coord2_rs']:
        tup = coord
        coords2[0].append(tup[0])
        coords2[1].append(tup[1])
        
    return [(int(np.round(np.average(coords1[0]))),int(np.round(np.average(coords1[1])))),(int(np.round(np.average(coords2[0]))),int(np.round(np.average(coords2[1]))))]
        

In [151]:
def get_coords(o):
    h , t = os.path.split(o)
    #print(v2_meta_df[v2_meta_df['file_name'] == t]['coord1'])
    #check size:
    if v2_graded[v2_graded['file_name'] == t]['coord1_rs'].shape[0]  > 1:
        #average the coordinates:
        print(v2_graded[v2_graded['file_name'] == t]['coord1_rs'])
        res = average_coords(v2_graded[v2_graded['file_name'] == t])
        return torch.tensor([[res[0][1],res[0][0]], [res[1][1],res[1][0]]] )
        
    c1 = v2_graded[v2_graded['file_name'] == t]['coord1_rs'].item()
    c2 = v2_graded[v2_graded['file_name'] == t]['coord2_rs'].item()
 
    #fast ai expects a format of y, x
    return torch.tensor([[c1[1],c1[0]],[c2[1],c2[0]] ])
    

In [148]:
diameter_files = parse_files(DATA_DIR / 'diameter')
diameter_files = [os.path.basename(f) for f in diameter_files]

In [ ]:
coord_dict = { o:get_coords(o) for o in diameter_files}

In [ ]:
pkl.dump(coord_dict, open(WORKING_DIR / 'coord_dict.pkl','rb'))

In [159]:
def get_label_pnt(o):
    h , t = os.path.split(o)
    return coord_dict[t]

In [185]:
src = (PointsItemList.from_folder(DATA_DIR / 'diameter')).split_by_rand_pct(seed=42).label_from_func(get_label_pnt)
tfms = get_transforms(max_lighting = 0.25)
size= (800,1040)
data = src.transform(tfms=tfms,tfm_y = True, size=size, resize_method=ResizeMethod.SQUISH).databunch(num_workers=num_cpu).normalize(imagenet_stats)

### training

In [191]:
learn = cnn_learner(data,models.resnet34, pretrained = True)

In [192]:
class MSELossFlat(nn.MSELoss): 
#“Same as `nn.MSELoss`, but flattens input and target.”
    def forward(self, input:Tensor, target:Tensor) -> Rank0Tensor:
        return super().forward(input.view(-1), target.view(-1))

In [193]:
learn.data.batch_size = 32

In [ ]:
learn.lr_find()

## building model with just diamater data.

## Some other stuff

In [266]:
disc_columns = ['DISC_visit_0_mean','DISC_visit_1_mean']
disc_df = uk_df[(~np.isnan(uk_df[disc_columns[0]])) | (~np.isnan(uk_df[disc_columns[1]]))]

In [ ]:
multi_image =[]
for fid in disc_df['FID']:
    if len(parse_files(UNZIPPED_DIR,str(fid))) > 1:
        multi_image.append(fid)
        break

In [339]:
right = [f for f in list(v2_meta_df['file_name']) if '21016' in f]

In [305]:
f_list = parse_files(UNZIPPED_DIR)

In [306]:
f_names = [f[-21:] for f in f_list]

In [313]:
fids = list(disc_df['FID'])

In [350]:
v2_meta_df['radius'] = v2_meta_df.apply(lambda x: cal_radius(x.coord1, x.coord2), axis=1)


In [353]:
v2_graded = v2_meta_df[v2_meta_df['radius'] > 0]

In [360]:
v2_graded_average  = v2_graded.groupby('file_name').mean()

In [368]:
ff = parse_files(UNZIPPED_DIR)

In [402]:
len(ff)

104342

In [369]:
for f in ff:
    shutil.copy(f, DATA_DIR / 'all')

## Preliminary Inference

In [364]:
path_img = UNZIPPED_DIR


path_img

In [379]:
v2_graded_average.reset_index(inplace = True)

In [401]:
src = ImageList.from_df(v2_graded_average, path = DATA_DIR / "all").split_by_rand_pct(seed=42).label_from_df(label_cls=FloatList)
tfms = get_transforms(max_lighting = 0.25) # or tfms=None if none are needed
size= (800,1040) # size=(224,224) or (400,224)
data = src.transform(tfms=tfms, size=size, resize_method=ResizeMethod.SQUISH).databunch(num_workers=num_cpu).normalize(imagenet_stats)

In [38]:
class MSELossFlat(nn.MSELoss): 
#“Same as `nn.MSELoss`, but flattens input and target.”
    def forward(self, input:Tensor, target:Tensor) -> Rank0Tensor:
        return super().forward(input.view(-1), target.view(-1))

In [119]:
learn.data.batch_size = 128

In [115]:
class L1LossFlat(nn.L1Loss):
#Mean Absolute Error Loss”
    def forward(self, input:Tensor, target:Tensor) -> Rank0Tensor:
        return super().forward(input.view(-1), target.view(-1))

## Processing cropping old


In [303]:
ukbb_image_folders_left =[
 'raw_image_21015_0_0_png',
 'raw_image_21015_0_1_png',
 'raw_image_21015_1_0_png',
]

ukbb_image_folders_right =[
'raw_image_21016_1_0_png',
 'raw_image_21016_0_0_png',
 'raw_image_21016_0_1_png']

In [28]:
ukbb_images_left = {dir_name:parse_files(UKBB_DIR / dir_name) for dir_name in ukbb_image_folders_left}
ukbb_images_right = {dir_name:parse_files(UKBB_DIR / dir_name) for dir_name in ukbb_image_folders_right}

In [26]:
list_files = parse_files(CROP_DIR / 'crop_image_21016_0_1_png')

In [24]:
ukbb_image_folders_right[2]

'raw_image_21016_0_1_png'

In [25]:
#rightimages left = 800, left images: left = 350
left = 800
top = 350
right = 1040 + left
bottom = 800 +top
save_path = CROP_DIR / 'crop_image_21016_0_1_png'
crop_files(ukbb_images_right[ukbb_image_folders_right[2]],(left,top,right,bottom),save_path, num_cpu)

100%|██████████| 686/686 [00:00<00:00, 305932.22it/s]


In [27]:
len(list_files)

686

In [34]:
## delete all files not supposed to be in
list_files_0_1 = parse_files(CROP_DIR / 'crop_image_21015_0_1_png')